# Visualising PAG neurons in CCF space
In this notebook we will load the .csv file containing the metadata from our PAG_scRNAseq project and use the CCF coordinates obtained after registration with Sharp-Track to visualise our sequenced cells with Brainrender.

### 1 | Import packages and set defaults

In [1]:
import brainrender
from brainrender.scene import Scene, DualScene, MultiScene
from brainrender.colors import makePalette, get_random_colors
from vedo import embedWindow
embedWindow(False)  # Disables scene embedding in jupyter notebooks
from brainrender.animation.video import BasicVideoMaker as VideoMaker
import pandas as pd # used to load the cwv

# // BRAINRENDER DEFAULT SETTINGS //
# Change some of the default settings
brainrender.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.BACKGROUND_COLOR = "white" # color of the background window (defaults to "white", try "blackboard")
brainrender.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.4, 0.4, 0.4])
brainrender.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
brainrender.DEFAULT_SCREENSHOT_NAME = "PAG_aspirated_cells" # screenshots will have this name and the time at which they were taken
brainrender.DEFAULT_SCREENSHOT_TYPE = ".png" # png, svg or jpg supported
brainrender.DEFAULT_SCREENSHOT_SCALE = 1 # values >1 yield higher resolution screenshots
brainrender.SCREENSHOT_TRANSPARENT_BACKGROUND = True # whether to save screenshots with transparent background

# // SET PARAMETERS //
# Save folder
save_folder = "D:/Dropbox (UCL - SWC)/Project_transcriptomics/analysis/PAG_scRNAseq_brainrender/output"

# Create screenshot parameters
screenshot_params = dict(folder = save_folder, name = "PAG_scRNAseq_brainrender")

### 2 | Load metadata including CCF coordinates
Other options to load registered points include `add_cells_from_file` and `get_probe_points_from_sharptrack`.

In [2]:
# Load data
pag_data = pd.read_csv("D:\\Dropbox (UCL - SWC)\\Project_transcriptomics\\analysis\\PAG_scRNAseq_brainrender\\PAG_scRNAseq_metadata_200617.csv")

# Look at the first 5 rows of the metadata
pag_data.head()

,Unnamed: 0,sequencing.id,date.collection,sample.id,sample.number,mouse.id,mouse.sex,mouse.age,mouse.singlehousedays,cell.id,...,RT.machine,RT.time,preamp.date,preamp.machine,preamp.time,date.purification,date.nanodrop,date.qubit,date.bioanalyzer,batch.sequencing_round
0,1,GC.OP.7836.EBR,180518,dmpag_vglut2_180518_c20_EBR,20,CE51.2,F,7.0,0,EBR,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
1,2,GC.OP.7836.EBT,180518,dmpag_vglut2_180518_c22_EBT,22,CE51.2,F,7.0,0,EBT,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
2,3,GC.OP.7836.EBU,180518,dlpag_vglut2_180518_c23_EBU,23,CE51.2,F,7.0,0,EBU,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
3,4,GC.OP.7836.EBV,180518,lpag_vglut2_180518_c24_EBV,24,CE51.2,F,7.0,0,EBV,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
4,5,GC.OP.7836.ECZ,180607,dlpag_vglut2_180607_c6_ECZ,6,BJ162.2,M,7.3,0,ECZ,...,classic,22:00,180608,non-classic,14:17,180609,180609.0,180609.0,180609.0,1


In [3]:
# List all column names in data
pag_data.columns

Index(['Unnamed: 0', 'sequencing.id', 'date.collection', 'sample.id',
       'sample.number', 'mouse.id', 'mouse.sex', 'mouse.age',
       'mouse.singlehousedays', 'cell.id', 'cell.type', 'cell.fluorophore',
       'patch.recorded', 'pipette.RNAseOUT', 'slice.number', 'slice.depth',
       'CCF.AllenAP', 'CCF.AllenDV', 'CCF.AllenML', 'CCF.AllenIndex',
       'CCF.AllenArea', 'CCF.PaxinosIndex', 'CCF.PaxinosArea',
       'PAG.hemisphere', 'PAG.PaxinosAPmanual', 'PAG.areacollection',
       'PAG.areamanualregistration', 'PAG.arearegistration', 'time.aspiration',
       'time.slicing', 'time.sinceslicingexact', 'time.sinceslicinghour',
       'batch.processing', 'RT.oligo', 'RT.date', 'RT.machine', 'RT.time',
       'preamp.date', 'preamp.machine', 'preamp.time', 'date.purification',
       'date.nanodrop', 'date.qubit', 'date.bioanalyzer',
       'batch.sequencing_round'],
      dtype='object')

#### 2.1 | Scaling up coordinates

In [4]:
# Inspect the CCF coordinates for each cell
pag_data[["cell.id", "cell.type", "PAG.areamanualregistration", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

,cell.id,cell.type,PAG.areamanualregistration,CCF.AllenAP,CCF.AllenDV,CCF.AllenML
0,EBR,VGluT2,dmpag,993,235,549
1,EBT,VGluT2,dmpag,993,231,594
2,EBU,VGluT2,dlpag,990,242,618
3,EBV,VGluT2,vlpag,993,302,525
4,ECZ,VGluT2,dlpag,939,250,525
...,...,...,...,...,...,...
511,WIBC,VGAT,dmpag,921,251,592
512,WIBD,VGAT,dmpag,921,247,580
513,WIBE,VGAT,dmpag,921,241,552
514,WIBF,VGAT,dmpag,993,229,592


In [5]:
# Scale up data
scale_factor = 10 # sharptrack uses a 10um reference atlas so the coordinates need to be scaled to match brainrender's

pag_data["CCF.AllenAP"] *= scale_factor
pag_data["CCF.AllenDV"] *= scale_factor
pag_data["CCF.AllenML"] *= scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

,cell.id,cell.type,CCF.AllenAP,CCF.AllenDV,CCF.AllenML
0,EBR,VGluT2,9930,2350,5490
1,EBT,VGluT2,9930,2310,5940
2,EBU,VGluT2,9900,2420,6180
3,EBV,VGluT2,9930,3020,5250
4,ECZ,VGluT2,9390,2500,5250
...,...,...,...,...,...
511,WIBC,VGAT,9210,2510,5920
512,WIBD,VGAT,9210,2470,5800
513,WIBE,VGAT,9210,2410,5520
514,WIBF,VGAT,9930,2290,5920


### 3 | Selecting a subset of cells
We can start by using the metadata to select subsets of cells. This will be useful to either just render some of the cells, or to color cells based on some metadata info.

The CCF coordinates were obtained by registering images to the Allen Brain Atlas using Sharp-Track (see Shamash et al. bioRxiv 2018 and https://github.com/cortex-lab/allenCCF), which yields coordinates at a resolution of 10 micrometers. In the Allen Brain Atlas, a point at coordinates \[1, 0, 0\] is at 10um from the origin (in other words, 1 unit of the atlas space equals 10um). However, BrainRender's space is at 1um resolution, so the first thing we need to do is to scale up the coordinate values by 10 to get them to match correctly.

In [ ]:
brainrender.SHOW_AXES = True
scene = Scene()
scene.render() # make sure to press 'Esc' to close not 'q' or kernel diesscene.render(interactive = True, video = False, camera = "sagittal", zoom = 1)

In [ ]:
# We can subset cells using any criteria we want. For instance, let's keep only those cells coming from female mice
cells_hemisphere_right = pag_data.loc[pag_data["PAG.hemisphere"] == "right"]
cells_hemisphere_left = pag_data.loc[pag_data["PAG.hemisphere"] == "left"]
cells_hemisphere_right.head()

In [ ]:
# We can also use multiple criteria at the same time
vgat_cells_hemisphere_left = pag_data.loc[(pag_data["PAG.hemisphere"] == "left")&(pag_data["cell.type"] == "VGAT")]
vgat_cells_hemisphere_left.head()

### 4 | Rendering cells with Brainrender
Now that we can select the data appropriately, we can render our cells in Brainrender and colour them according to any metadata we want. We will prepare different renderings in each of the following code chunks.

#### 4.1 | Rendering VGAT and VGluT2 cells separately

In [ ]:
brainrender.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Let's color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
# Create a scene with no title. You can also use scene.add_text to add other text elsewhere in the scene
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
# Set add_root = False to focus only on the desired brain area

# // ADD CELLS//
scene.add_cells(vgat_cells, color = "salmon", col_names = column_names, res = 24)
scene.add_cells(vglut2_cells, color = "skyblue", col_names = column_names, res = 24)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

#### 4.2 | Rendering cells from each hemisphere separately

In [ ]:
scene = Scene()
scene.add_cells(cells_hemisphere_right, color = "salmon", col_names = column_names, alpha = 1, radius = 25, res = 24)
scene.add_cells(cells_hemisphere_left, color = "skyblue", col_names = column_names, alpha = 1, radius = 25, res = 24)

scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # make sure to press 'Esc' to close not 'q' or kernel dies

#### 4.3 | Coloring cells by metadata (can't speficy the colors for now)

In [ ]:
# Another (easier) way to color cells based on some metadata attribute is as follows. However, at the moment this doesn't allow us to decide which color is assigned to each category
scene = Scene()
scene.add_cells( # Renders cells given their coordinates as a collection of spheres.
    pag_data, # pandas dataframe containing x,y,z coordinates
    color = "salmon", # str, color of spheres used to render the cells (Default value = "red"). Alternatively a list of colors specifying the color of each cell.
    color_by_region = None, # bool. If true the cells are colored according to the color of the brain region they are in
    color_by_metadata = "PAG.areamanualregistration", # str, if the name of a column of the coords dataframe is passed, cells are colored according to their value for that column. 
    radius = 25, # int, radius of spheres used to render the cells (Default value = 25)
    res = 24, # int, resolution of spheres used to render the cells (Default value = 3)
    alpha = 1, # float, transparency of spheres used to render the cells (Default value = 1)
    col_names = column_names, # ist of strings with names of pandas dataframe columns. If passed it should be a list of 3 columns which have the x, y, z coordinates. If not passed, it is assumed that the columns are ['x', 'y', 'z']
    regions=None, # if a list of brain regions acronym is passed, only cells in these regions will be added to the scene
    verbose=True,
    )

scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1)

#### 4.4 | Rendering VGAT and VGluT2 cells separately

In [ ]:
brainrender.SHADER_STYLE = "cartoon"
brainrender.BACKGROUND_COLOR = "white"

# // CREATE SCENE //
# Create a scene with no title. You can also use scene.add_text to add other text elsewhere in the scene
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
# Set add_root = False to focus only on the desired brain area

# // ADD BRAIN REGIONS //
# Add brain regions with scene.add_brain_regions
pag = scene.add_brain_regions(["PAG"], 
    alpha = 0.2, color = "darkgoldenrod", add_labels = False, use_original_color = False, wireframe = False)
dorsal_raphe = scene.add_brain_regions(["DR"], 
    alpha = 0.2, color = "olivedrab", add_labels = False, use_original_color = False, wireframe = True)
superior_colliculus = scene.add_brain_regions(["SCdg", "SCdw", "SCig", "SCiw", "SCm", "SCop", "SCs", "SCsg", "SCzo"],
    alpha = 0.2, color = "olivedrab", add_labels = False, use_original_color = False, wireframe = True)

# // ADD CELLS//
scene.add_cells(pag_data, color = "darkseagreen", col_names = column_names, color_by_metadata = "CCF.PaxinosArea",
                res = 24) # res -> resolution of spheres


# # // ADD LABELS //
# # Add labels with scene.add_actor_label
# scene.add_actor_label(pag, "PAG", size = 400, color = "blackboard", xoffset = 0, yoffset = 0, radius = None)
# scene.add_actor_label(superior_colliculus, "SC", size = 400, color = "blackboard", xoffset = 0, yoffset = 0, radius = None)
# scene.add_actor_label(dorsal_raphe, "DRN", size = 400, color = "blackboard", xoffset = 0, yoffset = 0, radius = None)


# # // CUT SCENE IN HALF AT DESIRED PLANE //
# # Cut scene in half (set showplane = True if you want to see the plane location)
# scene.cut_actors_with_plane("sagittal", showplane = False)
# # Or cut PAG along the midline
# scene.cut_actors_with_plane("sagittal", showplane = False, actors = pag) 


# # // EXPORT AS HTML //
# # Export to a .html file that can be opened in a browser to render an interactive brainrender scene
# scene.export_for_web("output/PAG_scRNAseq_brainrender.html") # you can pass a filepath where to save the scene


# # // MAKE VIDEO //
# # Create an instance of VideoMaker with our scene
# vm = VideoMaker(scene,
#     save_fld = save_folder, # folder where to save video
#     save_name = "PAG_video_scRNAseq_brainrender", # video name
#     #video_format = "mp4", # defaults to mp4
#     #duration = 3, # video duration in seconds (defaults to 3)
#     #niters = 60, # number of iterations (frames) when creating the video (defaults to 60)
#     #fps = 30 # framerate of video (defaults to 30)
#     ) 

# # Make a video and specify how the scene rotates at each frame. You can also change several parameters (destination folder, video name, fps, duration, etc.) specified above
# vm.make_video(azimuth = 1, elevation = 1, roll = 0, # rotation in degrees per frame on the relative axis    
#     save_fld = save_folder, save_name = "PAG_video_scRNAseq_brainrender",
#     video_format = "avi", duration = 5, niters = 50, fps = 30)


# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

### 5 | Dual scenes \[work in progress\]

In [ ]:
# Let's color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
# Create a scene with no title. You can also use scene.add_text to add other text elsewhere in the scene
scene_vgat = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
scene_vglut2 = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)

# // CREATE DUAL SCENE //
scene_dual = DualScene[scene_vgat, scene_vglut2]

# // ADD CELLS//
scene_vgat.add_cells(vgat_cells, color = "salmon", col_names = column_names, res = 24)
scene_vglut2.add_cells(vglut2_cells, color = "skyblue", col_names = column_names, res = 24)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene_dual.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

### 6 | Use enhanced Franklin-Paxinos atlas (Chon et al. 2019)

Using the enhanced atlas we can use the registered coordinates to obtain the PAG subdivision each cell belongs to.

In [ ]:
# Run this to get a list of the available atlases:
import brainatlas_api
from brainatlas_api import list_atlases
list_atlases()

In [1]:
# To use the atlas with PAG subdivisions, try:
# kim_unified_50um_v0.1
# kim_unified_25um_v0.1

scene = Scene(display_inset = False, title = None, camera = "sagittal", atlas = 'kim_unified_50um_v0.1')
scene.add_brain_regions(["DLPAG", "DMPAG", "LPAG", "VLPAG"],
    colors = get_random_colors(4), use_original_colo = False)
    #colors = dict(DLPAG = "blackboard", DMPAG = "olivedrab", LPAG = "lightgreen", VLPAG = "red"), use_original_color = False)
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

NameError: name 'Scene' is not defined

In [ ]:
coordinates = pag_data[["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

# Try the following to get the PAG area from the coordinates.
cell_region = scene.atlas.get_structure_from_coordinates(coordinates)
print(cell_region)